In [1]:
"""
create table Clientes(
id			int,
created_at datetime,
first_name nvarchar(255),
last_name  nvarchar(255),
email nvarchar(255),
cell_phone nvarchar(255),
country nvarchar(255),
state nvarchar(255),
street nvarchar(255),
number  nvarchar(255),
additionals  nvarchar(255)
)

"""

'\ncreate table Clientes(\nid\t\t\tint,\ncreated_at datetime,\nfirst_name nvarchar(255),\nlast_name  nvarchar(255),\nemail nvarchar(255),\ncell_phone nvarchar(255),\ncountry nvarchar(255),\nstate nvarchar(255),\nstreet nvarchar(255),\nnumber  nvarchar(255),\nadditionals  nvarchar(255)\n)\n\n'

In [2]:

import pandas as pd
import pyodbc 

server = 'EDMILSON\MSSQL' # Substitua pelo nome do servidor SQL Server
database = 'Python'  # Substitua pelo nome do banco de dados
conexaoDB = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      f'SERVER={server};'
                      f'DATABASE={database};'
                      'Trusted_Connection=yes;'
                      #'charset=UTF-16'
                      
                      )

cursor = conexaoDB.cursor()   # criando cursor de comando 

In [3]:
dados = pd.read_csv(r"K:\Python\Arquivos\Origem\arquivos_csv\Clientes.csv",delimiter=',')
#str(dados.columns).replace("'","") 
dados.head(5)


,id,created_at,first_name,last_name,email,cell_phone,country,state,street,number,additionals
0,0,2017-11-01 14:45:41,Marta,Jesus,NaN,9 9102-7834,Brasil,Acre,NaN,NaN,Conjunto 16
1,1,2017-10-16 00:50:39,Luana,Almeida,NaN,9 7328-8718,Brasil,Rio Grande do Sul,Avenida 56 do Estado Rio Grande do Sul,989.0,Conjunto 17
2,2,2018-06-16 17:51:29,Frida,Mendes,frida@meu_email.com,9 5906-7552,Brasil,São Paulo,Avenida 59 do Estado São Paulo,534.0,NaN
3,3,2018-01-17 03:02:58,Daniela,Avelino,daniela@exemplo.com,9 4642-9486,Brasil,Mato Grosso,NaN,NaN,NaN
4,4,2018-08-06 07:24:16,Romário,Teixeira,NaN,9 3093-6522,Brasil,Bahia,Praça 56 do Estado Bahia,191.0,Apto 12


In [4]:
# Converter a coluna 'created_at' para o formato de data e hora
dados['created_at'] = pd.to_datetime(dados['created_at'])
dados['email'] = dados['email'].fillna('Sem registro')
dados['street'] = dados['street'].fillna('Sem Info')
dados['number'] = dados['number'].fillna('Sem Numero')
dados['additionals'] = dados['additionals'].fillna('Sem Info')


In [5]:
for coluna in dados.columns:
    print(f"Coluna: {coluna}, Tipo de dados: {dados[coluna].dtypes}")

Coluna: id, Tipo de dados: int64
Coluna: created_at, Tipo de dados: datetime64[ns]
Coluna: first_name, Tipo de dados: object
Coluna: last_name, Tipo de dados: object
Coluna: email, Tipo de dados: object
Coluna: cell_phone, Tipo de dados: object
Coluna: country, Tipo de dados: object
Coluna: state, Tipo de dados: object
Coluna: street, Tipo de dados: object
Coluna: number, Tipo de dados: object
Coluna: additionals, Tipo de dados: object


In [6]:
str(dados.columns).replace("'","")  # coluna de origem 

'Index([id, created_at, first_name, last_name, email, cell_phone,\n       country, state, street, number, additionals],\n      dtype=object)'

In [7]:
cursor.execute('truncate table  [Clientes]')   #executa tarefa de  apagar dados
cursor.commit()

In [8]:
# Exemplo de inserção usando tupla de parâmetros
for index, linha in dados.iterrows():   
    linha.email = str(linha.email)  # Converter para o tipo 'str' antes da inserção
    linha.country = str(linha.country)
    linha.state = str(linha.state)
    linha.street = str(linha.street)
    linha.number = str(linha.number)
    linha.additionals = str(linha.additionals)
    
    #linha.cell_phone = str(linha.cell_phone)
    cursor.execute("INSERT INTO [Clientes] (id, created_at,first_name, last_name,email,cell_phone,country, state,street, number, additionals) VALUES (?,?,?,?,?,?,?,?,?,?,?)",linha.id, linha.created_at,linha.first_name,
                   linha.last_name,linha.email,linha.cell_phone,linha.country,linha.state,linha.street,linha.number,linha.additionals)
cursor.commit()   # validar dados no SQL Server
cursor.close()    #Fechar Cursor
conexaoDB.close() #Fechar Conexao